# InstanceCMR - Repetitions Fitting
Now that relevant dependencies are specified and testing, we'll jump right into fitting the model to larger portions of the dataset. This time, we'll do the entire 20-item list length subset of the Murdock (1962) dataset.

## Load Data

In [4]:
from instance_cmr.datasets import prepare_repdata
from instance_cmr.model_fitting import icmr_rep_likelihood
from instance_cmr.model_fitting import icmr_rep_objective_function
from instance_cmr.model_fitting import visualize_rep_fit
from instance_cmr.models import InstanceCMR

trials, events, list_length, presentations, list_types, rep_data, subjects = prepare_repdata(
    '../../data/repFR.mat')

events.head()

,subject,list,item,input,output,study,recall,repeat,intrusion,condition
0,1,1,0,1,1.0,True,True,0,False,4
1,1,1,1,2,2.0,True,True,0,False,4
2,1,1,2,3,3.0,True,True,0,False,4
3,1,1,3,4,4.0,True,True,0,False,4
4,1,1,4,5,5.0,True,True,0,False,4


## Fitting With Echo-Based Sensitivity Parameter

In [5]:
from scipy.optimize import differential_evolution
from numba.typed import List
import numpy as np

free_parameters = [
    'encoding_drift_rate',
    'start_drift_rate',
    'recall_drift_rate',
    'shared_support',
    'item_support',
    'learning_rate',
    'primacy_scale',
    'primacy_decay',
    'stop_probability_scale',
    'stop_probability_growth',
    'choice_sensitivity']

lb = np.finfo(float).eps
ub = 1-np.finfo(float).eps

bounds = [
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, 100),
    (lb, 100),
    (lb, ub),
    (lb, 10),
    (1, 10)
]

In [ ]:
# cost function to be minimized
# ours scales inversely with the probability that the data could have been 
# generated using the specified parameters and our model
for condition_index, selection in enumerate([list_types == 1, list_types > 1, list_types > 0]):

    cost_function = icmr_rep_objective_function(
            trials[selection], presentations[selection], list_types[selection], list_length,
             {'context_sensitivity': 1, 'feature_sensitivity': 1}, free_parameters)

    result = differential_evolution(cost_function, bounds, disp=True)
    print(result)

differential_evolution step 1: f(x)= 38789.5
differential_evolution step 2: f(x)= 34444.9
differential_evolution step 3: f(x)= 29701.2
differential_evolution step 4: f(x)= 23469.7
differential_evolution step 5: f(x)= 23352.4
differential_evolution step 6: f(x)= 23352.4
differential_evolution step 7: f(x)= 22746.8
differential_evolution step 8: f(x)= 22128.9
differential_evolution step 9: f(x)= 22040.2
differential_evolution step 10: f(x)= 22040.2
differential_evolution step 11: f(x)= 21878.7
differential_evolution step 12: f(x)= 21877.6
differential_evolution step 13: f(x)= 21877.6
differential_evolution step 14: f(x)= 21822.2
differential_evolution step 15: f(x)= 21702.1
differential_evolution step 16: f(x)= 21702.1
differential_evolution step 17: f(x)= 21702.1
differential_evolution step 18: f(x)= 21702.1
differential_evolution step 19: f(x)= 21702.1
differential_evolution step 20: f(x)= 21663.4
differential_evolution step 21: f(x)= 21663.4
differential_evolution step 22: f(x)= 19344

1:
```
     fun: 18893.699623102842
     jac: array([ -2.6411726 ,  26.22655303,  41.66249689,  -5.70616976,
         6.89069562,  -7.58045644,   0.81890896,  -0.11823431,
        -2.05327524,   6.80774975, -32.23576625])
 message: 'Optimization terminated successfully.'
    nfev: 14136
     nit: 71
 success: True
       x: array([9.27680726e-01, 2.19052136e-01, 9.96823218e-01, 2.57065752e-04,
       5.44460212e-01, 2.37385091e-01, 2.38541093e+01, 9.60950231e-01,
       2.35458005e-02, 9.14176255e-02, 5.20733974e-01])
```

2,3,4:
```
     fun: 41705.95666110226
     jac: array([-14.48352115,  -3.5157427 , -97.52548018, 127.94771465,
        14.22886271, -26.20945451,  -1.47992979,   0.        ,
        52.31922841, -31.09962565,  -4.51400408])
 message: 'Optimization terminated successfully.'
    nfev: 9219
     nit: 42
 success: True
       x: array([8.76094526e-01, 8.52373452e-01, 9.70242081e-01, 1.88866325e-04,
       3.98399638e-02, 3.24525612e-02, 2.38791407e+00, 6.66023782e+01,
       2.19539367e-02, 1.20933358e-01, 9.80864346e-01])
```

1,2,3,4:
```
     fun: 60731.76861567584
     jac: array([ 1.79899506e+02,  2.07095581e+02, -7.38997183e+01,  9.18937876e+03,
       -3.99745477e+02,  2.86886643e+02,  3.04571588e+00,  0.00000000e+00,
        3.54426447e+01,  2.16765329e+01,  5.13064149e+01])
 message: 'Optimization terminated successfully.'
    nfev: 10917
     nit: 48
 success: True
       x: array([8.85729103e-01, 8.64309296e-01, 9.72950128e-01, 9.69878669e-05,
       2.12559236e-02, 2.29115877e-02, 2.12408541e+00, 6.16905440e+01,
       2.51658967e-02, 1.01416770e-01, 9.63953204e-01])
```

In [6]:
for subject in np.unique(subjects):
    for condition_index, condition in enumerate([list_types == 1, list_types > 1, list_types > 0]):
    
        print(subject, condition_index)
        selection = np.logical_and(condition, subjects == subject)
        cost_function = icmr_rep_objective_function(
            trials[selection], presentations[selection], list_types[selection], list_length,
            {'context_sensitivity': 1, 'feature_sensitivity': 1}, free_parameters)

        result = differential_evolution(cost_function, bounds, disp=True)
        
        print(result)

1 0
differential_evolution step 1: f(x)= 1033.6
differential_evolution step 2: f(x)= 977.043
differential_evolution step 3: f(x)= 977.043
differential_evolution step 4: f(x)= 977.043
differential_evolution step 5: f(x)= 977.043
differential_evolution step 6: f(x)= 900.264
differential_evolution step 7: f(x)= 900.264
differential_evolution step 8: f(x)= 879.565
differential_evolution step 9: f(x)= 879.565
differential_evolution step 10: f(x)= 879.565
differential_evolution step 11: f(x)= 848.283
differential_evolution step 12: f(x)= 848.283
differential_evolution step 13: f(x)= 848.283
differential_evolution step 14: f(x)= 848.283
differential_evolution step 15: f(x)= 848.283
differential_evolution step 16: f(x)= 758.945
differential_evolution step 17: f(x)= 758.945
differential_evolution step 18: f(x)= 758.945
differential_evolution step 19: f(x)= 758.945
differential_evolution step 20: f(x)= 758.945
differential_evolution step 21: f(x)= 758.945
differential_evolution step 22: f(x)= 75

## Trace-Based Sensitivity

In [3]:
from scipy.optimize import differential_evolution
from numba.typed import List
import numpy as np

free_parameters = [
    'encoding_drift_rate',
    'start_drift_rate',
    'recall_drift_rate',
    'shared_support',
    'item_support',
    'learning_rate',
    'primacy_scale',
    'primacy_decay',
    'stop_probability_scale',
    'stop_probability_growth',
    'context_sensitivity']

lb = np.finfo(float).eps
ub = 1-np.finfo(float).eps

bounds = [
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, ub),
    (lb, 100),
    (lb, 100),
    (lb, ub),
    (lb, 10),
    (lb, 10)
]

In [11]:
# cost function to be minimized
# ours scales inversely with the probability that the data could have been 
# generated using the specified parameters and our model
for condition_index, selection in enumerate([list_types == 1, list_types > 1]): #, list_types > 0

    cost_function = icmr_rep_objective_function(
            trials[selection], presentations[selection], list_types[selection], list_length,
             {'choice_sensitivity': 1, 'feature_sensitivity': 1}, free_parameters)

    result = differential_evolution(cost_function, bounds, disp=True)
    print(result)

differential_evolution step 1: f(x)= 37124.5
differential_evolution step 2: f(x)= 32693.2
differential_evolution step 3: f(x)= 27901.6
differential_evolution step 4: f(x)= 22056.2
differential_evolution step 5: f(x)= 21903
differential_evolution step 6: f(x)= 21903
differential_evolution step 7: f(x)= 21903
differential_evolution step 8: f(x)= 21644.5
differential_evolution step 9: f(x)= 21644.5
differential_evolution step 10: f(x)= 21644.5
differential_evolution step 11: f(x)= 21644.5
differential_evolution step 12: f(x)= 21644.5
differential_evolution step 13: f(x)= 21644.5
differential_evolution step 14: f(x)= 21644.5
differential_evolution step 15: f(x)= 21644.5
differential_evolution step 16: f(x)= 21454.7
differential_evolution step 17: f(x)= 20979.8
differential_evolution step 18: f(x)= 20979.8
differential_evolution step 19: f(x)= 20979.8
differential_evolution step 20: f(x)= 20979.8
differential_evolution step 21: f(x)= 20979.8
differential_evolution step 22: f(x)= 20979.8
   

In [4]:
for subject in np.unique(subjects):
    for condition_index, condition in enumerate([list_types > 0]):  # list_types == 1, list_types > 1
    
        print(subject, condition_index)
        selection = np.logical_and(condition, subjects == subject)
        cost_function = icmr_rep_objective_function(
            trials[selection], presentations[selection], list_types[selection], list_length,
            {'choice_sensitivity': 1, 'feature_sensitivity': 1}, free_parameters)

        result = differential_evolution(cost_function, bounds, disp=True)
        
        print(result)

1 0
differential_evolution step 1: f(x)= 5227.65
differential_evolution step 2: f(x)= 3303.44
differential_evolution step 3: f(x)= 3155.21
differential_evolution step 4: f(x)= 2969.98
differential_evolution step 5: f(x)= 2709.26
differential_evolution step 6: f(x)= 2600.69
differential_evolution step 7: f(x)= 2600.69
differential_evolution step 8: f(x)= 2600.69
differential_evolution step 9: f(x)= 2600.69
differential_evolution step 10: f(x)= 2583.9
differential_evolution step 11: f(x)= 2583.9
differential_evolution step 12: f(x)= 2579.42
differential_evolution step 13: f(x)= 2517.08
differential_evolution step 14: f(x)= 2517.08
differential_evolution step 15: f(x)= 2517.08
differential_evolution step 16: f(x)= 2517.08
differential_evolution step 17: f(x)= 2473.21
differential_evolution step 18: f(x)= 2354.8
differential_evolution step 19: f(x)= 2288.58
differential_evolution step 20: f(x)= 2288.58
differential_evolution step 21: f(x)= 2288.58
differential_evolution step 22: f(x)= 2095